# Práctico 2

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Red Neuronal Recurrente Long Short Term Memory](https://en.wikipedia.org/wiki/Long_short-term_memory) (LSTM).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

La razón por la que se escoigió una Red Neuronal Recurrente obedece a que la misma tiene aplicaciones en el Procesamiento del Lenguaje Natural.

En particular, se utiliza una RNN del tipo **many to one** debido a que la entrada es una secuencia de palabras tokenizadas y codificadas, y la salida en un valor entero correspondiente a la etiqueta de una categoría. Cabe aclarar también que NO se trata de una RNN bidireccional porque usa información soalmente de la izquierda del contexto. Como utiliza la información procesada del trabajo práctico 1, se usan secuencias cortas (longitud 17 para el set de entrenamiento).

## Importaciones

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm, trange
from sklearn.metrics import balanced_accuracy_score
from practico1_modulo import *

## Constantes

In [2]:
EPOCHS = 3
BATCH_SIZE = 100

## Carga de datos

Carga de datos de entrenamiento. Son los mismos datos que se utilizaron para el Trabajo Práctico 1. Para más información consultar archivo [Practico_1.md](https://github.com/grobiglio/deepleaning/blob/master/practico/Practico_1.md#deep-learning---trabajo-pr%C3%A1ctico-1).

In [3]:
X_train = torch.load('./data/X_train.pt')
# y_train = torch.load('./data/y_train.pt')
y_train = torch.tensor(torch.load('./data/y_train.pt'), dtype=torch.float)

/tmp/ipykernel_2329329/1027201965.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(torch.load('./data/y_train.pt'), dtype=torch.float)


In [4]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
X_train = X_train[:1000000]
X_train.shape

torch.Size([1000000, 17])

In [5]:
# La reducción del dataset de entrenamiento es temporal
# Cuando compruebe que funciona se eliminará esta celda.
y_train = y_train[:1000000]
y_train.shape

torch.Size([1000000])

Carga de datos de prueba

In [6]:
X_test = torch.load('./data/X_test.pt')
# y_test = torch.load('./data/y_test.pt')
y_test = torch.tensor(torch.load('./data/y_test.pt'), dtype=torch.float)

/tmp/ipykernel_2329329/2269913238.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(torch.load('./data/y_test.pt'), dtype=torch.float)


In [7]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
X_test = X_test[:500000]
X_test.shape

torch.Size([500000, 16])

In [8]:
# La reducción del dataset de prueba es temporal.
# Cuando compruebe que funciona se eliminará esta celda.
y_test = y_test[:500000]
y_test.shape

torch.Size([500000])

## Embedding de títulos

In [9]:
# https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding
embeddings_matrix = torch.load('./data/embeddings_matrix.pt')
embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                          padding_idx=0)

## Construcción del Dataset

In [10]:
train_dataset = MeLiChallengeDataset(X_train, y_train)
test_dataset = MeLiChallengeDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          drop_last=False)
i = 0
for data in tqdm(train_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de entrenamiento.')

test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         drop_last=False)
i = 0
for data in tqdm(test_loader):
    i += 1
print(f'Recorrida exitosa de {i} batches de prueba.')

  0%|          | 0/10000 [00:00<?, ?it/s]

Recorrida exitosa de 10000 batches de entrenamiento.


  0%|          | 0/5000 [00:00<?, ?it/s]

Recorrida exitosa de 5000 batches de prueba.


## Construcción del Modelo

In [13]:
class MeLiChallengeLSTM(nn.Module):
    def __init__(self, embeddings):
        super(MeLiChallengeLSTM, self).__init__()
        self.embeddings = embeddings
        output_size = 1
        self.lstm_config = {'input_size': 300,
                            'hidden_size': 150, # tamaño de la capa oculta
                            'num_layers': 1,
                            'bias': True,
                            'batch_first': True,
                            'dropout': 0,
                            'bidirectional': False,
                            'proj_size': 0}
        
        # Set our fully connected layer parameters
        self.linear_config = {'in_features': 150,
                              'out_features': output_size,
                              'bias': True}
        
        # Instanciate the layers
        # Documentación LSTM 👉 https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.lstm = nn.LSTM(**self.lstm_config)
        self.classification_layer = nn.Linear(**self.linear_config)
        self.activation = nn.Sigmoid()

    def forward(self, inputs):
        emb = self.embeddings(inputs)
        lstm_out, _ = self.lstm(emb) # guardo el estado y descarto el contexto
        lstm_out = lstm_out[:, -1, :].squeeze() # guardo el último estado
        predictions = self.activation(self.classification_layer(lstm_out))
        return predictions

In [14]:
modelo = MeLiChallengeLSTM(embeddings)
print(modelo)

MeLiChallengeLSTM(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (lstm): LSTM(300, 150, batch_first=True)
  (classification_layer): Linear(in_features=150, out_features=1, bias=True)
  (activation): Sigmoid()
)


## Algoritmo de Optimización

In [15]:
learning_rate = 0.001
loss_function = nn.BCELoss()
optimizer = optim.Adam(modelo.parameters(), learning_rate)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Utilizando {device}')
modelo.to(device)

Utilizando cuda


MeLiChallengeLSTM(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (lstm): LSTM(300, 150, batch_first=True)
  (classification_layer): Linear(in_features=150, out_features=1, bias=True)
  (activation): Sigmoid()
)

## Entrenamiento y evaluación del modelo

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    '''Entrenamiento de una red neuronal.
    
    Parámetros:
    -----------
    - dataloader: Iterador (objeto) de Pytorch construido en base al dataset basado en la clase MeLiChallengeDataset.
    - model: Modelo (objeto) basado en la clase MeLiChallengeClassifier.
    - loss_fn: Función de costo.
    - optimizer: Optimizador.
    
    Salidas:
    --------
    train_loss: Valor promedio de la función de costo minimizados de cada uno de los batches.
    
    '''
    size = len(dataloader.dataset)
    model.train()
    running_loss = []
    for batch, data in enumerate(tqdm(dataloader)):
        X, y = data['data'].to(device), data['target'].to(device)
        optimizer.zero_grad()
        pred = model(X) # Esto devuelve siempre un tensor de unos, por eso no funciona 😡
        loss = loss_fn(pred.squeeze(), y) # Aquí se compara un tensor de unos con los valores verdaderos
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())
        train_loss = sum(running_loss) / len(running_loss)
        
    return train_loss

In [18]:
def test(dataloader, model, loss_fn):
    '''Evaluación de una red neuronal.
    
    Parámetros:
    -----------
    - dataloader: Iterador (objeto) de Pytorch construido en base al dataset basado en la clase MeLiChallengeDataset.
    - model: Modelo (objeto) basado en la clase MeLiChallengeClassifier.
    - loss_fn: Función de costo.
    
    Salidas:
    --------
    - train_loss: Valor promedio de la función de costo minimizados de cada uno de los batches.
    - avp: Precisión.
    '''
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    running_loss = []
    targets = []
    predictions = []
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in tqdm(dataloader):
            X, y = data['data'].to(device), data['target'].to(device)
            pred = model(X) # Esto devuelve siempre un tensor de unos, por eso no funciona 😡
            running_loss.append(loss_function(pred.squeeze(), y).item())
            targets.extend(y.cpu().detach().numpy())
            predictions.extend(pred.squeeze().cpu().round().detach().numpy())
            
        test_loss = sum(running_loss) / len(running_loss)
        avp = balanced_accuracy_score(targets, predictions)
                                    
    return test_loss, avp

In [19]:
history = {
    'train_loss': [],
    'test_loss': [],
    'test_avp': []
}

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_loader, modelo, loss_function, optimizer)
    print("\t Final train_loss", train_loss)
    history['train_loss'].append(train_loss)
    test_loss, avp = test(test_loader, modelo, loss_function)
    print("\t Final test_loss", test_loss)
    print("\t Final test_avp", avp)
    history['test_loss'].append(test_loss)
    history['test_avp'].append(avp)
print("!Listo!")

Epoch 1
-------------------------------


  0%|          | 0/10000 [00:00<?, ?it/s]

	 Final train_loss -30997.030854624772


  0%|          | 0/5000 [00:00<?, ?it/s]

	 Final test_loss -31219.0474
	 Final test_avp 0.0015822784810126582
Epoch 2
-------------------------------


...

	 Final test_loss -31219.0474
	 Final test_avp 0.0015822784810126582
Epoch 3
-------------------------------


  0%|          | 0/10000 [00:00<?, ?it/s]

	 Final train_loss -31227.0784


  0%|          | 0/5000 [00:00<?, ?it/s]

	 Final test_loss -31219.0474
	 Final test_avp 0.0015822784810126582
!Listo!


## ¿Por qué no funciona?

No funciona porque el modelo devuelve siempre un tensor de unos. Evidentemente hay un problema con el modelo.

Logramos entender el funcionamiento de la RNN LSTM pero no logramos que aún funcione.